In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import savetxt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import AveragePooling1D
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("Solar.csv", names=['Irradiance'])

In [4]:
data.head()

,Irradiance
0,0.00
1,0.00
2,0.00
3,3.76
4,21.72


In [5]:
datamat = data['Irradiance'].to_numpy()
datamat = datamat.reshape(len(datamat),1)

In [6]:
trainval = int(np.floor(0.75*len(datamat)))
train_data = datamat[0:trainval]
test_data = datamat[trainval:]

In [7]:
#Split into input ouput sets
def split_seq(seq, n_steps):
    X, y = list(), list()
    for i in range(len(seq)):
    
        #Find end of seq
        end_ix = i+n_steps
        #Check if sequence passed
        if end_ix>len(seq)-1:
            break

        #Gather input and ouputs
        seq_x, seq_y = seq[i:end_ix], seq[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [8]:
#No. of steps
n_steps = 4

#Split into inputs, outputs for training data
Xtrain, Ytrain = split_seq(train_data, n_steps)

n_features = 1
Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1], n_features)

#Split into input and outputs for test data
Xtest, Ytest = split_seq(test_data, n_steps)

Xtest = Xtest.reshape(Xtest.shape[0], Xtest.shape[1], n_features)

In [9]:
#Define Model
model = tf.keras.Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model.add(AveragePooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [10]:
#Fit model
hist = model.fit(Xtrain, Ytrain, epochs=50, batch_size=64, validation_split=0.15, verbose=0)

In [ ]:
a = hist.history['loss']
a = np.array(a)
a = a.reshape(len(a),1)
plt.plot(a)

In [ ]:
# plot history
plt.plot(hist.history['loss'], label='train')
# plt.plot(hist.history['val_loss'], label='val')
# plt.legend()
# plt.show()

In [30]:
#Preparing Entire dataset for the network
X, Y = split_seq(datamat, n_steps)

In [31]:
yhat_test = model.predict(Xtest, verbose=0)
yhat = model.predict(X, verbose=0)

In [32]:
Yout_test = np.hstack((Ytest, yhat_test))
Yout = np.hstack((Y, yhat))

In [33]:
savetxt('80_4step_yout_test64_3.csv', Yout_test, delimiter=',')
savetxt('80_4stepnorm_yout64_3.csv', Yout, delimiter=',')